# load data

In [1]:
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse
import requests
from datetime import date
from datetime import timedelta

In [2]:
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
    
from pipeline_functions import load_tweets

NameError: name 'sys' is not defined

## load tweets older then two weeks

In [28]:
df_tweets = load_tweets("../home.db", 21)

# utils

In [29]:
df_tweets.tail()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
47699,1349143306261782532,28623584,RT @SerenaMalyon: Seeing all of your work this...,2021-01-12T23:56:44+00:00,en,1349120805637025796,N/A,0,None
47700,1349143755278848004,28623584,"RT @nezumiyasha: #PortfolioDay I'm Nez, freela...",2021-01-12T23:58:31+00:00,en,1349116778828472324,N/A,0,None
47701,1349143840435924992,28623584,"RT @JanainaArt: Hi #portfolioday I'm Janaina, ...",2021-01-12T23:58:52+00:00,en,1349060159591641089,N/A,0,None
47702,1349143921801134081,28623584,"RT @ZackRock: Hey #PortifolioDay, it’s me, Zac...",2021-01-12T23:59:11+00:00,en,1349115167448199168,N/A,0,None
47703,1349143989010661377,28623584,RT @LaurenBlakeArt: It's the first 2021 #Portf...,2021-01-12T23:59:27+00:00,en,1349021226052808705,N/A,0,None


In [33]:
def find_twitter(tweet):
    twitter_url = re.findall(r"twitter.com/\S+", tweet)
    if len(twitter_url) > 0:
        return twitter_url
    else:
        return 0

In [34]:
find_twitter(df_tweets.iloc[47].full_text)[-1].split("/status/")[1]

'697066672998449152'

In [35]:
find_twitter(df_tweets.iloc[47656].full_text)

0

In [40]:
df_tweets["twitter_links"] =  df_tweets.full_text.apply(find_twitter)

In [15]:
# qt_without_status = df_tweets[(df_tweets.is_quote_status == 1) & (df_tweets.quoted_status == 0)].id.tolist()

In [16]:
asdf = df_tweets[df_tweets.retweeted_status > 0].retweeted_status.value_counts()

In [17]:
asdf.describe()

count    16167.000000
mean         1.021711
std          0.179931
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          6.000000
Name: retweeted_status, dtype: float64

In [18]:
len(asdf[asdf > 1])

290

In [20]:
df_tweets[df_tweets.retweeted_status == 1339969022549487616]

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,twitter_links
27092,1341955209979899910,916052708930588672,RT @circleboom: Why should I delete tweets abo...,2020-12-24T03:53:49+00:00,en,1339969022549487616,0,0,None,0
28715,1342864505093705728,916052708930588672,RT @circleboom: Why should I delete tweets abo...,2020-12-26T16:07:01+00:00,en,1339969022549487616,0,0,None,0
35938,1345468530205065218,916052708930588672,RT @circleboom: Why should I delete tweets abo...,2021-01-02T20:34:29+00:00,en,1339969022549487616,0,0,None,0
53831,1351334760136769538,916052708930588672,RT @circleboom: Why should I delete tweets abo...,2021-01-19T01:04:48+00:00,en,1339969022549487616,0,0,None,0
55093,1351646941860536320,916052708930588672,RT @circleboom: Why should I delete tweets abo...,2021-01-19T21:45:17+00:00,en,1339969022549487616,0,0,None,0
59437,1352604904951607298,916052708930588672,RT @circleboom: Why should I delete tweets abo...,2021-01-22T13:11:54+00:00,en,1339969022549487616,0,0,None,0


In [19]:
asdf[:50]

1339969022549487616    6
1345610869149937664    6
1317485210896896000    5
1338933782452314112    5
1341542597177864192    4
1336421397883711488    4
1334920409624563712    4
1327008283769630720    4
1339257875034566656    4
1354600999835607040    4
1352284823486754816    4
1349819426799960064    4
1341463081004752896    4
1330216172537733120    4
1341783834287878144    4
1354547622133051392    3
1320715294210428928    3
1347872328013778944    3
1335707643156062208    3
1345130408170541056    3
1304481380110397440    3
1353691134682345472    3
1345568606973792256    3
1341479406104162304    3
1353850378438070272    3
1352429784115122176    3
1352319762253156352    3
1351313539151982592    3
1313176921522479104    3
1352289036854046720    3
1335976036174073856    3
1355497971538939904    3
1351522763681443840    3
1348131824367783936    3
1201271295297622016    3
1237423058861916160    3
1334958223540019200    3
1354099325996048384    3
1355074685566939136    3
1344032210077298688    3


In [ ]:
def news_in_qt_rt(df):
    #TODO refractor this
    df["all_news"] = df["contains_news"].copy()

    df_qt = df[["id", "contains_news"]].copy()
    df_qt.columns = ["quoted_status", "qt_news"]
    df = df.merge(df_qt, on="quoted_status", how="left")
    df["qt_news"] = df["qt_news"].fillna(0).astype(np.int64)
    df["all_news"] = df["qt_news"].astype(np.int64) + df["contains_news"].astype(np.int64)

    df_qt = df[["id", "contains_news"]].copy()
    df_qt.columns = ["in_reply_to_status_id", "rt_news"]
    df = df.merge(df_qt, on="in_reply_to_status_id", how="left")
    df["rt_news"] = df["rt_news"].fillna(0).astype(np.int64)
    df["all_news"] = df["rt_news"].astype(np.int64) + df["all_news"].astype(np.int64)
    
    return df

In [ ]:
def find_url(tweet):
    url = re.findall(r"http\S+", tweet)
    return url


def clean_links(tweet):
    tweet = re.sub(r"bit.ly/\S+", "", tweet)
    tweet = re.sub(r"t.co/\S+", "", tweet)
    tweet = re.sub(r"buff.ly/\S+", "", tweet)
    tweet = re.sub(r"https://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://twitter.com/\S+", "", tweet)
    return tweet

def remove_tw_urls(tweet):
    tweet = re.sub(r"https://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://twitter.com/\S+", "", tweet)
    return tweet

def expand_link(url):
    if url in ["https://", "http://"]:
        return url
    else:
        try:
            session = requests.Session()
            resp = session.head(url, allow_redirects=True)
            long_url = resp.url
            return long_url
        except:
            return url

def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
    if domain == "twitter.com":
        pass
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return domain


def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return l

In [ ]:
def find_news(df, news_domains_list):
    df["clean_text"] = df["full_text"].apply(remove_tw_urls)
    df["urls"] = df["clean_text"].apply(find_url)
    df["urls"] = df.urls.apply(lambda x: [clean_links(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)

    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

def drop_contains(df, column_name, word_list):
    for string in word_list:
        df["lower"] = df["full_text"].str.lower()
        df = df[df["lower"].str.contains(string)]
        df.drop(["lower"], axis=1, inplace=True)
    return df

# searching for not-expanded news urls

In [ ]:
dfz = df_tweets.copy()

In [ ]:
with open("news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())
dfz = find_news(dfz, news_domains)

In [ ]:
dfz["has_domain"] = dfz.domains.apply(lambda x: len(x))

In [ ]:
dfz.head()

In [ ]:
# dfz["has_domain"] = dfz.domains.apply(lambda x: 1 if len(x) > 0 else 0)
dfz["has_domain"] = dfz.domains.apply(lambda x: len(x))

In [ ]:
dfz = dfz[dfz.has_domain > 0]
dfz.shape

In [ ]:
dfz = dfz[dfz.contains_news == 0]
dfz.shape

In [ ]:
dfz.head()

In [ ]:
domain_list = dfz.domains.tolist()

In [ ]:
domains = []
for i in domain_list:
    for d in i:
        if d in domains:
            pass
        else:
            domains.append(d)

In [ ]:
dfz["urls_exp"] = 0

In [ ]:
dfz.head()

In [ ]:
# import json
# short_url =[]
# for c, i in enumerate(dfz.urls):
#     print(c)
#     if (c + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e = expand_link(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
#     with open('short_url', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))



In [ ]:
with open('short_url') as json_file:
    short_url = json.load(json_file)

In [ ]:
short_url[1]

In [ ]:
# expand_link(url)

In [ ]:
dfz.urls[921:922]

In [ ]:
# x = 922
# for c, i in enumerate(dfz.urls[x:]):
#     print(c+x, i)
#     if (c + x + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e = expand_link(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
#     with open('short_url', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[1100:1101]

In [ ]:
# x = 1101
# for c, i in enumerate(dfz.urls[x:]):
#     print(c+x, i)
#     if (c + x + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e = expand_link(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
#     with open('short_url', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[3295:3296]

In [ ]:
# x = 3296
# for c, i in enumerate(dfz.urls[x:]):
#     print(c+x, i)
#     if (c+x+1) % 100 == 0:
#         print(f"{(c+x+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e = expand_link(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
#     with open('short_url', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[5652:5653]

In [ ]:
x = 5653
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c+x+1) % 100 == 0:
        print(f"{(c+x+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
    with open('short_url', 'w') as file:
        file.write(json.dumps(short_url, indent=4))